In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import os
from datetime import datetime, timedelta

from google.colab import drive

LOADED = True

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Specify the folder path in Google Drive
folder_path = '/content/drive/MyDrive/mimic_csvs'

# Individual csv paths
admissions_path = os.path.join(folder_path, 'ADMISSIONS.csv')
icu_stays_path = os.path.join(folder_path, 'ICUSTAYS.csv')
patients_path = os.path.join(folder_path, 'PATIENTS.csv')
chart_events_path = os.path.join(folder_path, 'CHARTEVENTS.csv')
complete_data_path = os.path.join(folder_path, 'mimic_complete_data.csv')

Mounted at /content/drive


In [2]:
# Model parameters and outcome
# hadm_id as primary key to predict by hosptial visit rather than by patient

if not LOADED:
  data = {
    'hadm_id': [],
    'subject_id': [],
    'gender': [],
    'age': [],
    'marital_status': [],
    'ethnicity': [],
    'diagnosis': [],
    'vitals': [],
    'icu_admitted': []
  }


In [3]:
# Load hadm_id, subject_id, ethnicity, marital_status, diagnosis from ADMISSIONS.CSV

if not LOADED:
  if not os.path.exists(admissions_path):
    raise FileNotFoundError(f"File not found: {admissions_path}")

  admissions_df = pd.read_csv(admissions_path)
  data['hadm_id'] = admissions_df['HADM_ID'].tolist()
  data['ethnicity'] = admissions_df['ETHNICITY'].tolist()
  data['diagnosis'] = admissions_df['DIAGNOSIS'].tolist()
  data['marital_status'] = admissions_df['MARITAL_STATUS'].tolist()
  data['subject_id'] = admissions_df['SUBJECT_ID'].tolist()

  row_count = total_data_points = len(data['hadm_id'])

  # Load gender, age (at time of visit) from PATIENTS.CSV
  if not os.path.exists(patients_path):
    raise FileNotFoundError(f"File not found: {patients_path}")

  patients_df = pd.read_csv(patients_path)
  for i in range(row_count):
    subject_id = data['subject_id'][i]
    hadm_id = data['hadm_id'][i]

    gender = patients_df[patients_df['SUBJECT_ID'] == subject_id]['GENDER'].values[0]
    data['gender'].append(gender)

    dob = patients_df[patients_df['SUBJECT_ID'] == subject_id]['DOB'].values[0]
    admittime = admissions_df[admissions_df['HADM_ID'] == hadm_id]['ADMITTIME'].values[0]

    dob_dt = datetime.strptime(str(dob)[:10], '%Y-%m-%d')
    admit_dt = datetime.strptime(str(admittime)[:10], '%Y-%m-%d')

    age = (admit_dt - dob_dt).days // 365  # Convert days to years

    data['age'].append(age)

  # Load ICU stays
  if not os.path.exists(icu_stays_path):
    raise FileNotFoundError(f"File not found: {icu_stays_path}")

  icustays_df = pd.read_csv(icu_stays_path)
  icu_hadm_ids = set(icustays_df['HADM_ID'].tolist())

  # Check if each hadm_id is in the ICU stays
  data['icu_admitted'] = [hadm_id in icu_hadm_ids for hadm_id in data['hadm_id']]

In [4]:
# Load Vitals Data - First 24 hours after hospital admission time
# Each data['vitals'] entry format = [heart_rate_mean, sbp_mean, dbp_mean, temp_mean, resp_rate_mean, spo2_mean]

if not LOADED:
  if not os.path.exists(chart_events_path):
    raise FileNotFoundError(f"File not found: {chart_events_path}")

  data['vitals'] = [None] * row_count

  # Vital sign ITEMIDs corresponding to required vitals parameters
  vital_itemids = {
      'heart_rate': [211, 220045],
      'systolic_bp': [51, 442, 455, 6701, 220179, 220050],
      'diastolic_bp': [8368, 8440, 8441, 8555, 220180, 220051],
      'temp': [223761, 678],
      'resp_rate': [618, 615, 220210, 224690],
      'spo2': [646, 220277]
  }

  # Flatten all vital ITEMIDs into one list
  all_vital_ids = [item for items in vital_itemids.values() for item in items]

  # Load CHARTEVENTS in chunks
  # Filter rows by keeping only parameter vital types and hadm_ids in data
  chunk_size = 500000
  filtered_chunks = []

  print("Loading CHARTEVENTS in chunks...")
  for i, chunk in enumerate(pd.read_csv(chart_events_path, chunksize=chunk_size)):
      print(f"Processing chunk {i+1}...")

      # Keep only rows with parameter vital type ITEMIDs
      chunk_filtered = chunk[chunk['ITEMID'].isin(all_vital_ids)]

      # Keep only rows for hadm_ids in data
      chunk_filtered = chunk_filtered[chunk_filtered['HADM_ID'].isin(data['hadm_id'])]
      filtered_chunks.append(chunk_filtered)

  # Combine all filtered chunks
  vitals_df = pd.concat(filtered_chunks, ignore_index=True)
  print(f"Loaded {len(vitals_df)} vital sign records")

  # Convert times to datetime format
  vitals_df['CHARTTIME'] = pd.to_datetime(vitals_df['CHARTTIME'])
  admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])

  # Merge admission times onto vitals (one-time operation)
  vitals_df = vitals_df.merge(
      admissions_df[['HADM_ID', 'ADMITTIME']],
      on='HADM_ID',
      how='left'
  )

  # Filter to first 24 hours
  vitals_df['time_diff_hours'] = (vitals_df['CHARTTIME'] - vitals_df['ADMITTIME']).dt.total_seconds() / 3600
  vitals_df_24hr = vitals_df[(vitals_df['time_diff_hours'] >= 0) & (vitals_df['time_diff_hours'] <= 24)]

  # Map ITEMIDs to vital names
  itemid_to_vital = {}
  for vital_name, item_ids in vital_itemids.items():
      for item_id in item_ids:
          itemid_to_vital[item_id] = vital_name

  vitals_df_24hr['vital_type'] = vitals_df_24hr['ITEMID'].map(itemid_to_vital)

  # Group and calculate means (vectorized)
  vitals_summary = vitals_df_24hr.groupby(['HADM_ID', 'vital_type'])['VALUENUM'].mean().unstack(fill_value=None)

  # Ensure columns are in correct order
  vital_order = ['heart_rate', 'systolic_bp', 'diastolic_bp', 'temp', 'resp_rate', 'spo2']
  vitals_summary = vitals_summary.reindex(columns=vital_order)

  # Fill data['vitals'] in correct order
  for i in range(row_count):
      hadm_id = data['hadm_id'][i]
      if hadm_id in vitals_summary.index:
          data['vitals'][i] = vitals_summary.loc[hadm_id].tolist()
      else:
          data['vitals'][i] = [None] * 6

In [5]:
def safe_eval(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else [None]*6
    except:
        return [None]*6

In [6]:
# View Data Frame and save complete dataset to CSV

if not LOADED:
  df = pd.DataFrame(data)
  df.to_csv(complete_data_path, index=False)
  print('data saved to mimic_complete_data.csv')
else:
  df = pd.read_csv(complete_data_path)
  df['vitals'] = df['vitals'].apply(safe_eval)
  print('data loaded from mimic_complete_data.csv')

total_data_points = len(df)
column_count = len(df.columns)
print(f"Total number of data points: {total_data_points}\n\nData Frame:")
print(df)

data loaded from mimic_complete_data.csv
Total number of data points: 58976

Data Frame:
       hadm_id  subject_id gender  age marital_status ethnicity  \
0       165315          22      F   64        MARRIED     WHITE   
1       152223          23      M   71        MARRIED     WHITE   
2       124321          23      M   75        MARRIED     WHITE   
3       161859          24      M   39         SINGLE     WHITE   
4       129635          25      M   58        MARRIED     WHITE   
...        ...         ...    ...  ...            ...       ...   
58971   191113       98800      F   19         SINGLE     WHITE   
58972   101071       98802      F   83        WIDOWED     WHITE   
58973   122631       98805      M   42        MARRIED     WHITE   
58974   170407       98813      F   60        MARRIED     WHITE   
58975   190264       98813      F   63        MARRIED     WHITE   

                                               diagnosis  \
0                                BENZODIAZEPIN

In [7]:
# Cleaning and Preprocess

# PHASE A: CLEANING

# 0) Handle missing main category data
df = df[df['diagnosis'].notna()]
df['marital_status'] = df['marital_status'].fillna('Not Disclosed') # Treat missing marital status as new category "Not Disclosed"

# 1) Convert list of vitals into separate columns
vitals_df = pd.DataFrame(df['vitals'].tolist(),
                         columns=['hr_mean', 'sbp_mean', 'dbp_mean',
                                 'temp_mean', 'rr_mean', 'spo2_mean'],
                         index=df.index)

# 2) Create a singular X (the inputs) and y (the target)
X = pd.concat([df[['age', 'gender', 'marital_status', 'ethnicity', 'diagnosis']], vitals_df], axis=1)
y = df['icu_admitted'].astype(int)

# 3) Split the data before statistical learning to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# PHASE B: PREPROCESSING

# 4) Process data differently based on their data type, this handles imputation, scaling, encoding, and vectorizing
preprocessor = ColumnTransformer(
    transformers=[
        # Impute vitals and then scale them
        ('vitals_num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), ['hr_mean', 'sbp_mean', 'dbp_mean', 'temp_mean', 'rr_mean', 'spo2_mean']),

        # Scale age
        ('age_num', StandardScaler(), ['age']),

        # One-Hot Encode categories
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['gender', 'marital_status', 'ethnicity']),

        # Vectorize diagnosis text
        ('text', TfidfVectorizer(max_features=100, stop_words='english'), 'diagnosis')
    ])

# 5) Fit on training, and transform both
X_train_final = preprocessor.fit_transform(X_train)
X_test_final = preprocessor.transform(X_test)

In [8]:
print(f"Row count after processing = {len(df)}")
for column in df.columns:
    if column == 'vitals':
      none_count = df[column].apply(lambda x: None in x if isinstance(x, list) else True).sum()
    else:
      none_count = df[column].isna().sum() + df[column].apply(lambda x: x is None).sum()
    print(f"{column}: {none_count} None entries")

Row count after processing = 58951
hadm_id: 0 None entries
subject_id: 0 None entries
gender: 0 None entries
age: 0 None entries
marital_status: 0 None entries
ethnicity: 0 None entries
diagnosis: 0 None entries
vitals: 23247 None entries
icu_admitted: 0 None entries
